<a href="https://colab.research.google.com/github/ronanmccormack-ca/ProgrammingforBigData/blob/main/Big%20Data%20Project/Yield%20Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.0.3/spark-3.0.3-bin-hadoop2.7.tgz
!tar xf spark-3.0.3-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["SPARK_HOME"] = "/content/spark-3.0.3-bin-hadoop2.7"
import findspark
import pandas as pd
from pyspark.sql.functions import *
findspark.init()

SyntaxError: ignored

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Yield Prediction")\
        .config('spark.ui.port','4050')\
        .getOrCreate()
spark

In [3]:
!git clone https://github.com/ronanmccormack-ca/ProgrammingforBigData.git

Cloning into 'ProgrammingforBigData'...
remote: Enumerating objects: 57, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 57 (delta 15), reused 35 (delta 3), pack-reused 0
Unpacking objects: 100% (57/57), done.


In [4]:
df = spark.read.csv('/content/ProgrammingforBigData/Big Data Project/data/inventory_data.csv',header='true',inferSchema=True)

In [142]:
df.groupBy("batch_id","work_order_type_name",'substance_type_name').sum('measure_event_value')\
.where((col("work_order_type_name") == 'Drying')&(col("substance_type_name")=='Dried Cannabis')).orderBy('batch_id').show()

+--------+--------------------+-------------------+------------------------+
|batch_id|work_order_type_name|substance_type_name|sum(measure_event_value)|
+--------+--------------------+-------------------+------------------------+
|    7246|              Drying|     Dried Cannabis|                   55700|
|    9697|              Drying|     Dried Cannabis|                  130662|
|   15527|              Drying|     Dried Cannabis|                   51596|
|   15528|              Drying|     Dried Cannabis|                    1230|
|   15529|              Drying|     Dried Cannabis|                    1148|
|   15530|              Drying|     Dried Cannabis|                    2028|
|   15531|              Drying|     Dried Cannabis|                     967|
|   15532|              Drying|     Dried Cannabis|                    1123|
|   15533|              Drying|     Dried Cannabis|                    1845|
|   15534|              Drying|     Dried Cannabis|                    2679|

In [144]:
df.select('batch_id','cultivar_id','work_order_type_name','work_order_close_date','substance_type_name','measure_event_value')\
.orderBy('measure_event_value')\
.filter((col('batch_id')==9697)\
        &(col("work_order_type_name")=='Drying')\
        &(col("substance_type_name")=='Dried Cannabis'))\
.show()

+--------+-----------+--------------------+---------------------+-------------------+-------------------+
|batch_id|cultivar_id|work_order_type_name|work_order_close_date|substance_type_name|measure_event_value|
+--------+-----------+--------------------+---------------------+-------------------+-------------------+
|    9697|       4494|              Drying|   2021-07-15 9:39:40|     Dried Cannabis|             130662|
+--------+-----------+--------------------+---------------------+-------------------+-------------------+



In [155]:
df.select('batch_id','cultivar_id','work_order_type_name','work_order_close_date','substance_type_name','measure_event_value')\
.filter((col('cultivar_id')==4494)\
        &(col("work_order_type_name")=='Drying')\
        &(col("substance_type_name")=='Dried Cannabis'))\
.groupBy('batch_id','work_order_close_date').sum('measure_event_value')\
.show()

+--------+---------------------+------------------------+
|batch_id|work_order_close_date|sum(measure_event_value)|
+--------+---------------------+------------------------+
|    9697|   2021-07-15 9:39:40|                  130662|
|   15527|  2021-11-24 13:48:12|                   51596|
+--------+---------------------+------------------------+



In [98]:
co2_sensor_1_data = spark.read.csv('/content/ProgrammingforBigData/Big Data Project/data/co2_sensor_1.csv',header='true',inferSchema=True)
lux_sensor_1_data = spark.read.csv('/content/ProgrammingforBigData/Big Data Project/data/lux_sensor_1.csv',header='true',inferSchema=True)
temp_hum_sensor_1_data = spark.read.csv('/content/ProgrammingforBigData/Big Data Project/data/temp_hum_sensor_1.csv',header='true',inferSchema=True)
co2_sensor_2_data = spark.read.csv('/content/ProgrammingforBigData/Big Data Project/data/co2_sensor_2.csv',header='true',inferSchema=True)
lux_sensor_2_data = spark.read.csv('/content/ProgrammingforBigData/Big Data Project/data/lux_sensor_2.csv',header='true',inferSchema=True)
temp_hum_sensor_2_data = spark.read.csv('/content/ProgrammingforBigData/Big Data Project/data/temp_hum_sensor_2.csv',header='true',inferSchema=True)
co2_sensor_3_data = spark.read.csv('/content/ProgrammingforBigData/Big Data Project/data/co2_sensor_3.csv',header='true',inferSchema=True)
lux_sensor_3_data = spark.read.csv('/content/ProgrammingforBigData/Big Data Project/data/lux_sensor_3.csv',header='true',inferSchema=True)
temp_hum_sensor_3_data = spark.read.csv('/content/ProgrammingforBigData/Big Data Project/data/temp_hum_sensor_3.csv',header='true',inferSchema=True)

In [158]:
lux_sensor_1_data.withColumn("ts",to_timestamp(col("Date"))) \
  .withColumn("datetype",to_date(col("ts"))) \
  .groupBy('datetype').agg(mean('Value').alias('lux_sensor_1_mean'))\
  .filter((col('datetype')<='2021-07-15')&(col('datetype')>='2021-04-15'))\
  .join(lux_sensor_2_data.withColumn("ts",to_timestamp(col("Date"))) \
  .withColumn("datetype",to_date(col("ts"))) \
  .groupBy('datetype').agg(mean('Value').alias('lux_sensor_2_mean'))\
  .filter((col('datetype')<='2021-07-15')&(col('datetype')>='2021-04-15'))\
  ,on='datetype')\
  .join(lux_sensor_3_data.withColumn("ts",to_timestamp(col("Date"))) \
  .withColumn("datetype",to_date(col("ts"))) \
  .groupBy('datetype').agg(mean('Value').alias('lux_sensor_3_mean'))\
  .filter((col('datetype')<='2021-07-15')&(col('datetype')>='2021-04-15'))\
  ,on='datetype')\
  .join(co2_sensor_1_data.withColumn("ts",to_timestamp(col("Date"))) \
  .withColumn("datetype",to_date(col("ts"))) \
  .groupBy('datetype').agg(mean('Value').alias('co2_sensor_1_mean'))\
  .filter((col('datetype')<='2021-07-15')&(col('datetype')>='2021-04-15'))\
  ,on='datetype')\
  .join(co2_sensor_2_data.withColumn("ts",to_timestamp(col("Date"))) \
  .withColumn("datetype",to_date(col("ts"))) \
  .groupBy('datetype').agg(mean('Value').alias('co2_sensor_2_mean'))\
  .filter((col('datetype')<='2021-07-15')&(col('datetype')>='2021-04-15'))\
  ,on='datetype')\
  .join(co2_sensor_3_data.withColumn("ts",to_timestamp(col("Date"))) \
  .withColumn("datetype",to_date(col("ts"))) \
  .groupBy('datetype').agg(mean('Value').alias('co2_sensor_3_mean'))\
  .filter((col('datetype')<='2021-07-15')&(col('datetype')>='2021-04-15'))\
  ,on='datetype')\
  .join(temp_hum_sensor_1_data.withColumn("ts",to_timestamp(col("Date"))) \
  .withColumn("datetype",to_date(col("ts"))) \
  .groupBy('datetype').agg(mean('Value').alias('temp_hum_sensor_1_mean'))\
  .filter((col('datetype')<='2021-07-15')&(col('datetype')>='2021-04-15'))\
  ,on='datetype')\
  .join(temp_hum_sensor_2_data.withColumn("ts",to_timestamp(col("Date"))) \
  .withColumn("datetype",to_date(col("ts"))) \
  .groupBy('datetype').agg(mean('Value').alias('temp_hum_sensor_2_mean'))\
  .filter((col('datetype')<='2021-07-15')&(col('datetype')>='2021-04-15'))\
  ,on='datetype')\
  .join(temp_hum_sensor_3_data.withColumn("ts",to_timestamp(col("Date"))) \
  .withColumn("datetype",to_date(col("ts"))) \
  .groupBy('datetype').agg(mean('Value').alias('temp_hum_sensor_3_mean'))\
  .filter((col('datetype')<='2021-07-15')&(col('datetype')>='2021-04-15'))\
  ,on='datetype')\
  .sort('datetype',ascending=True)\
  .show(truncate=False)

+----------+------------------+------------------+------------------+-----------------+-----------------+------------------+----------------------+----------------------+----------------------+
|datetype  |lux_sensor_1_mean |lux_sensor_2_mean |lux_sensor_3_mean |co2_sensor_1_mean|co2_sensor_2_mean|co2_sensor_3_mean |temp_hum_sensor_1_mean|temp_hum_sensor_2_mean|temp_hum_sensor_3_mean|
+----------+------------------+------------------+------------------+-----------------+-----------------+------------------+----------------------+----------------------+----------------------+
|2021-04-15|2860.8819444444443|2232.8611111111113|5179.694444444444 |860.5347222222222|894.6875         |525.7222222222222 |58.956388888888846    |69.34562500000001     |68.77645833333332     |
|2021-04-16|2912.1111111111113|2309.909722222222 |5201.263888888889 |838.2291666666666|874.8472222222222|576.7847222222222 |58.103333333333325    |63.56611111111109     |64.71840277777781     |
|2021-04-17|2895.784722222222 

In [159]:
lux_sensor_1_data.withColumn("ts",to_timestamp(col("Date"))) \
  .withColumn("datetype",to_date(col("ts"))) \
  .groupBy('datetype').agg(mean('Value').alias('lux_sensor_1_mean'))\
  .filter((col('datetype')<='2021-11-24')&(col('datetype')>='2021-07-24'))\
  .join(lux_sensor_2_data.withColumn("ts",to_timestamp(col("Date"))) \
  .withColumn("datetype",to_date(col("ts"))) \
  .groupBy('datetype').agg(mean('Value').alias('lux_sensor_2_mean'))\
  .filter((col('datetype')<='2021-11-24')&(col('datetype')>='2021-07-24'))\
  ,on='datetype')\
  .join(lux_sensor_3_data.withColumn("ts",to_timestamp(col("Date"))) \
  .withColumn("datetype",to_date(col("ts"))) \
  .groupBy('datetype').agg(mean('Value').alias('lux_sensor_3_mean'))\
  .filter((col('datetype')<='2021-11-24')&(col('datetype')>='2021-07-24'))\
  ,on='datetype')\
  .join(co2_sensor_1_data.withColumn("ts",to_timestamp(col("Date"))) \
  .withColumn("datetype",to_date(col("ts"))) \
  .groupBy('datetype').agg(mean('Value').alias('co2_sensor_1_mean'))\
  .filter((col('datetype')<='2021-11-24')&(col('datetype')>='2021-07-24'))\
  ,on='datetype')\
  .join(co2_sensor_2_data.withColumn("ts",to_timestamp(col("Date"))) \
  .withColumn("datetype",to_date(col("ts"))) \
  .groupBy('datetype').agg(mean('Value').alias('co2_sensor_2_mean'))\
  .filter((col('datetype')<='2021-11-24')&(col('datetype')>='2021-07-24'))\
  ,on='datetype')\
  .join(co2_sensor_3_data.withColumn("ts",to_timestamp(col("Date"))) \
  .withColumn("datetype",to_date(col("ts"))) \
  .groupBy('datetype').agg(mean('Value').alias('co2_sensor_3_mean'))\
  .filter((col('datetype')<='2021-11-24')&(col('datetype')>='2021-07-24'))\
  ,on='datetype')\
  .join(temp_hum_sensor_1_data.withColumn("ts",to_timestamp(col("Date"))) \
  .withColumn("datetype",to_date(col("ts"))) \
  .groupBy('datetype').agg(mean('Value').alias('temp_hum_sensor_1_mean'))\
  .filter((col('datetype')<='2021-11-24')&(col('datetype')>='2021-07-24'))\
  ,on='datetype')\
  .join(temp_hum_sensor_2_data.withColumn("ts",to_timestamp(col("Date"))) \
  .withColumn("datetype",to_date(col("ts"))) \
  .groupBy('datetype').agg(mean('Value').alias('temp_hum_sensor_2_mean'))\
  .filter((col('datetype')<='2021-11-24')&(col('datetype')>='2021-07-24'))\
  ,on='datetype')\
  .join(temp_hum_sensor_3_data.withColumn("ts",to_timestamp(col("Date"))) \
  .withColumn("datetype",to_date(col("ts"))) \
  .groupBy('datetype').agg(mean('Value').alias('temp_hum_sensor_3_mean'))\
  .filter((col('datetype')<='2021-11-24')&(col('datetype')>='2021-07-24'))\
  ,on='datetype')\
  .sort('datetype',ascending=True)\
  .show(truncate=False)

+----------+--------------------+------------------+------------------+------------------+------------------+------------------+----------------------+----------------------+----------------------+
|datetype  |lux_sensor_1_mean   |lux_sensor_2_mean |lux_sensor_3_mean |co2_sensor_1_mean |co2_sensor_2_mean |co2_sensor_3_mean |temp_hum_sensor_1_mean|temp_hum_sensor_2_mean|temp_hum_sensor_3_mean|
+----------+--------------------+------------------+------------------+------------------+------------------+------------------+----------------------+----------------------+----------------------+
|2021-08-04|0.0                 |0.875             |0.0               |1087.0            |1237.9166666666667|1104.8032786885246|82.84506944444449     |83.16548611111108     |83.47888888888887     |
|2021-08-05|4.111888111888112   |5.489510489510489 |6.615384615384615 |1012.2916666666666|1161.6971830985915|1034.4965034965035|81.21013888888896     |81.20486111111111     |81.07583333333338     |
|2021-08-0